In [1]:
## Utilizando como gerenciador de pacotes o Conda
# !conda clean --all
# %conda update --all

## Utilizando como gerenciador de pacotes o PIP
# !python.exe -m pip install --upgrade pip
%pip install -U pip

# %pip install flash-attn --no-build-isolation

Note: you may need to restart the kernel to use updated packages.


In [2]:
import sys
import torch
import subprocess

print(f"\nDados do ambiente local")
# !nvcc --version
def verificar_versoes():
    """Verifica e exibe as versões  da GPU, do Python e bibliotecas CUDA instaladas no sistema."""

    # Versão do Python
    python_version = sys.version.split()[0]
    print(f"   Versão Python: {python_version}")
    print(f"  Versão PyTorch: {torch.__version__}")
    if torch.cuda.is_available():
        print(f"  Versão do CUDA: {torch.version.cuda}")
        print(f" Versão do cuDNN: {torch.backends.cudnn.version()}")
    else:
        print("CUDA não disponível, não está configurado corretamente.")

    # Versão do CUDA (usando nvcc)
    try:
        cuda_version = subprocess.check_output(["nvcc", "--version"])
        cuda_version_l3 = [' '.join(cuda_version.decode("utf-8").strip().split("\n")[-2].split(',')[1:])][0].strip()
        cuda_version_l4 = cuda_version.decode("utf-8").strip().split("\n")[-1]
        print(f"    CUDA Toolkit: {cuda_version_l3}")
        print(f"      Build nvcc: {cuda_version_l4}")
    except FileNotFoundError:
        print("nvcc não encontrado. CUDA pode não estar instalado ou configurado corretamente.")

# Chamando a função para verificar as versões
verificar_versoes()

print(f"\nDetalhes da biblioteca PyTorch")
!pip3 show torch


Dados do ambiente local
   Versão Python: 3.11.9
  Versão PyTorch: 2.2.2
CUDA não disponível, não está configurado corretamente.
    CUDA Toolkit: release 12.5  V12.5.40
      Build nvcc: Build cuda_12.5.r12.5/compiler.34177558_0

Detalhes da biblioteca PyTorch
Name: torch
Version: 2.2.2
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /home/mak/miniconda3/envs/rapids-24.08/lib/python3.11/site-packages
Requires: filelock, fsspec, jinja2, networkx, sympy, typing-extensions
Required-by: contextualSpellCheck, curated-transformers, EasyNMT, sentence-transformers, spacy-curated-transformers, spacy-transformers, torchaudio, torchvision, torchviz


# Execução 01: Sem mecanismo de atenção

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
import time

# Carregar modelo e tokenizer pré-treinados
model_name = "bert-base-uncased"  # Escolha o modelo desejado
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to("cuda")  # Mover para GPU

# Textos de entrada (exemplo)
texts = [
    "This is a sentence about natural language processing.",
    "Another sentence about NLP.",
    "An unrelated sentence about dogs."
]

# Pré-processamento
encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors="pt").to("cuda")

# Medir tempo de execução
start_time = time.time()

# Embedding
with torch.no_grad():
    embeddings = model(**encoded_input)["pooler_output"].cpu().numpy()

# Cálculo de similaridade
similarity_matrix = cosine_similarity(embeddings)

end_time = time.time()

print("Matriz de similaridade:")
print(similarity_matrix)

print(f"Tempo de execução: {end_time - start_time:.2f} segundos")

# Execução 02: Com mecanismo de atenção

A versão do CUDA precisa ser compatível com a Flash Attention. A Flash Attention foi introduzida no PyTorch 2.0 e requer CUDA 11.8 ou superior.

In [3]:
# !conda uninstall pytorch torchvision torchaudio

Channels:
 - pypi
 - conda-forge
 - pytorch
 - defaults
 - rapidsai
 - nvidia
 - pytorch-nightly
 - pyg
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /home/mak/miniconda3/envs/rapids-24.08

  removed specs:
    - pytorch
    - torchaudio
    - torchvision


The following packages will be REMOVED:

  aom-3.6.1-h59595ed_0
  blas-1.0-mkl
  cpuonly-2.0-0
  ffmpeg-4.4.2-gpl_hdf48244_113
  gmp-6.3.0-hac33072_2
  gmpy2-2.1.5-py311hc4f1f91_1
  gnutls-3.7.9-hb077bed_0
  lame-3.100-h166bdaf_1003
  libdrm-2.4.123-hb9d3cd8_0
  libidn2-2.3.7-hd590300_0
  libpciaccess-0.18-hd590300_0
  libtasn1-4.19.0-h166bdaf_0
  libunistring-0.9.10-h7f98852_0
  libva-2.21.0-h4ab18f5_2
  libvpx-1.13.1-h59595ed_0
  llvm-openmp-15.0.7-h0cdce71_0
  mpc-1.3.1-hfe3b2da_0
  mpfr-4.2.1-h38ae2d0_2
  mpmath-1.2.1-py311_0
  nettle-3.9.1-h7ab15ed_0
  openh264-2.3.1-hcb278e6_2
  p11-kit-0.24.1-hc5aa10d_0
  pyg-2.5.2-py311_torch_2.2.0_cpu
  pytorch-2.2.2-py3.11_cpu_0
  pytorch-mutex-1

## Criar o ambiente pelo Terminal:
### Sempre atualizar versões mais recentes em https://pytorch.org/

    conda create -n pytorch_env python=3.11
    conda activate pytorch_env
    conda install pytorch torchvision torchaudio pytorch-cuda=12.4 -c pytorch-nightly -c nvidia

In [ ]:
# !conda config --add channels conda-forge
# !conda config --add channels rapidsai
# !conda config --add channels nvidia


# %conda install -c rapidsai -c nvidia -c conda-forge pylibcugraph=24.6.1 cudatoolkit=12.5
# %conda install pytorch torchvision torchaudio pytorch-cuda=12.4 -c pytorch-nightly -c nvidia

In [1]:
# Chamando a função para verificar as versões
verificar_versoes()

print(f"\nDetalhes da biblioteca PyTorch")
!pip3 show torch

NameError: name 'verificar_versoes' is not defined

In [ ]:
import pandas as pd
import networkx as nx

url = "https://data.rapids.ai/cugraph/datasets/cit-Patents.csv"
df = pd.read_csv(url, sep=" ", names=["src", "dst"], dtype="int32")
G = nx.from_pandas_edgelist(df, source="src", target="dst")

%time result = nx.betweenness_centrality(G, k=10)

In [ ]:
df

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# Criar um grafo aleatório
G = nx.gnp_random_graph(10, 0.3)

# Definir um layout
pos = nx.spring_layout(G) # Outras opções: circular_layout, spectral_layout, etc.

# Desenhar o grafo com personalizações
nx.draw(G, pos, with_labels=True, node_color='skyblue', node_size=500, font_size=10, font_color='black', width=2, edge_color='gray')

# Mostrar o grafo
plt.show()

# Estimativas de tempo de execução 

Considerando detecção de características do grafo e recursos computacionais

In [ ]:
# Calcular e exibir o número de nós e arestas
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()
print("Número de nós:", num_nodes)
print("Número de arestas:", num_edges)

# Calcular e exibir a densidade do grafo
density = nx.density(G)
print("Densidade do grafo:", density)

# Obter informações de hardware
cpu_count = psutil.cpu_count(logical=True)
ram_gb = psutil.virtual_memory().total / (1024 ** 3)

# Estimar o tempo de execução (modelo simplificado)
k = 10  # Valor de k usado no cálculo da centralidade
estimated_time = (num_nodes * num_edges * density * k) / (cpu_count * ram_gb)

In [ ]:
# Executar o comando e medir o tempo de execução
# %time result = nx.betweenness_centrality(G, k=10)

# Exibir o tempo estimado e o tempo real de execução
print("Tempo estimado:", estimated_time)
print("Tempo real:", _ # A variável mágica '_' contém o tempo de execução do último comando %)

# Visualizar a distribuição da centralidade betweenness
betweenness_data = pd.DataFrame(list(result.items()), columns=['Node', 'Betweenness Centrality'])

chart = alt.Chart(betweenness_data).mark_bar().encode(
    x=alt.X('Node:N', sort='-y'),
    y=alt.Y('Betweenness Centrality:Q'),
    tooltip=['Node', 'Betweenness Centrality']
).properties(
    title='Distribuição da Centralidade Betweenness'
).interactive()

chart.save('betweenness_centrality_distribution.json')

In [ ]:
import pandas as pd
import networkx as nx
import timeit
import altair as alt

# Carregar o dataframe a partir da URL
url = "https://data.rapids.ai/cugraph/datasets/cit-Patents.csv"
df = pd.read_csv(url, sep=" ", names=["src", "dst"], dtype="int32")

# Criar o grafo a partir do dataframe
G = nx.from_pandas_edgelist(df, source="src", target="dst")

# Calcular e exibir o número de nós e arestas
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()
print("Número de nós:", num_nodes)
print("Número de arestas:", num_edges)

# Calcular e exibir a densidade do grafo
density = nx.density(G)
print("Densidade do grafo:", density)

# Executar o comando e medir o tempo de execução usando timeit
start_time = timeit.default_timer()
result = nx.betweenness_centrality(G, k=10)
end_time = timeit.default_timer()
actual_time = end_time - start_time

# Exibir o tempo real de execução
print("Tempo real:", actual_time)

# Visualizar a distribuição da centralidade betweenness
betweenness_data = pd.DataFrame(list(result.items()), columns=['Node', 'Betweenness Centrality'])

chart = alt.Chart(betweenness_data).mark_bar().encode(
    x=alt.X('Node:N', sort='-y'),
    y=alt.Y('Betweenness Centrality:Q'),
    tooltip=['Node', 'Betweenness Centrality']
).properties(
    title='Distribuição da Centralidade Betweenness'
).interactive()

chart.save('betweenness_centrality_distribution.json')

# Exibir a distribuição da centralidade betweenness em formato tabular, ordenada de forma decrescente
betweenness_data_sorted = betweenness_data.sort_values(by='Betweenness Centrality', ascending=False)
print(betweenness_data_sorted)

In [ ]:
pos = nx.spring_layout(G)  # Outras opções: circular_layout, spectral_layout, etc.
nx.draw(G, pos, with_labels=True)

In [ ]:
%conda install -c rapidsai -c nvidia pylibcugraph

In [ ]:
%pip3 install --extra-index-url https://pypi.nvidia.com/pylibcugraph-cu12 pylibcugraph_cu12-24.6.1-cp311-cp311-manylinux_2_28_x86_64.whl